REMOVE THIS BEFORE SUBMITTING: guys plz dont run this, its jus so the TA has it technically available

The below code will build your database. This code assumes you have installed 'magenta' via pip INSTEAD of anaconda.

Structure your files as follows:
```
<name of working directory>/
├─ lmd_wan/
│  ├─ eval/
│  │  ├─ original (contains midi files)/
│  ├─ test/
│  │  ├─ original (contains midi files)/
│  ├─ train/
│  │  ├─ original (contains midi files)/
├─ <other code files on this level>
├─ <such as config.py, data_processing.ipynb etc>
├─ <other project folders are here too>/
```

In [ ]:
!python3 .local/lib/python3.8/site-packages/magenta/scripts/convert_dir_to_note_sequences.py   \
    --input_dir='/path/to/lmd_wan/train/original'   \
    --output_file='path/to/lmd_wan/train/ns_original.tfrecord'   \
    --recursive

!python3 .local/lib/python3.8/site-packages/magenta/scripts/convert_dir_to_note_sequences.py   \
    --input_dir='path/to/lmd_wan/test/original'   \
    --output_file='path/to/lmd_wan/test/ns_original.tfrecord'   \
    --recursive

!python3 .local/lib/python3.8/site-packages/magenta/scripts/convert_dir_to_note_sequences.py   \
    --input_dir='path/to/lmd_wan/eval/original'   \
    --output_file='path/to/lmd_wan/eval/ns_original.tfrecord'   \
    --recursive

Next, mask songs for infill task

In [1]:
import encode_embeddings
folders = ['train', 'test', 'eval']
for f in folders:
    ns_input_path = '~/ECE324/lmd_wan/' + f + '/ns_original.tfrecord'
    masked_ns_output_path = '~/ECE324/lmd_wan/' + f + '/ns_masked.tfrecord'
    print(ns_input_path, masked_ns_output_path)
    encode_embeddings.mask_ns(ns_input_path, masked_ns_output_path)

/home/david/.local/lib/python3.8/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/home/david/.local/lib/python3.8/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


~/ECE324/lmd_wan/train/ns_original.tfrecord ~/ECE324/lmd_wan/train/ns_masked.tfrecord
Starting Pipe


Masking /id/midi/original/60e124d6e1e06a019ed62214e07c19b9cae0bb88::ae38599d9849bb974b678c4f950321c8.mid (13.599293)
Masking /id/midi/original/74386b8bc20b2240f651164a2f7c9987595f8caf::a1884df156e23176d8517c3f7f745b04.mid (269.177461)
Masking /id/midi/original/d71b2974b4ffd7d9368816ea1a10a0d989312bb4::ab059f773a3828a285db87e9a6118b27.mid (131.420772)
Masking /id/midi/original/cbd89fdb1e600ae8edcd6e9187df744a50e1bbca::7cdf303b5334c5b7fa4a3d3b4efd6a08.mid (170.113966)
Masking /id/midi/original/c591dd5fe5c7bf1d19de483eae074a15939aead3::a16aea82ece0e5bbaccf4fd24462204a.mid (279.666667)
Masking /id/midi/original/8c16f0b7b23abc10c9e7b50e53a50f49161a37f4::742b2fc83476d8dc0c8e8d2163c28a48.mid (263.912972)
Masking /id/midi/original/21687ece7689f59d05b5e1e6d1b45e6a031ca666::7153c9d6a1d089c57219ef858f7f94e3.mid (229.200956)
Masking /id/midi/original/bdf996704633250762fc95351731924723697de1::effac22ba2fadadbe5856e75d4de0bef.mid (248.813385)
Masking /id/midi/original/93767832444dd014743f30ea8f5ad8a

Next, encode training, testing, and evaluation

In [ ]:
import encode_embeddings
folders = ['eval', 'train']
type = ['original','masked']
for f in folders:
    for t in type:
        if f == 'train' and t == 'original': break 
        ns_input_path = '~/ECE324/lmd_wan/' + f + '/ns_' + t + '.tfrecord'
        encoded_ns_output_path = '~/ECE324/lmd_wan/' + f + '/encoded_' + t + '.tfrecord'
        print(ns_input_path, encoded_ns_output_path)
        encode_embeddings.encode_ns(ns_input_path, encoded_ns_output_path)

: 